In [3]:
from methods.llm import get_vllm_endpoint_func, get_claude_response
get_endpoint_response = get_vllm_endpoint_func("", "ppas13gg82lsdh")

Could not load vllm class, check CUDA support and GPU RAM size
:: Sending warmup message to initialize the server ...
:: Server initialized successfully!


In [12]:
!pip install langgraph

  Attempting uninstall: msgpack
    Found existing installation: msgpack 1.0.3
    Uninstalling msgpack-1.0.3:
      Successfully uninstalled msgpack-1.0.3
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [13]:
# Test on meta_api fuctionals
from methods.meta_api import nodes_from_api
api_name = "langgraph"
doc_nodes, guide_nodes = nodes_from_api(api_name, fast_response=get_endpoint_response, slow_response=get_claude_response, max_links=2) # Takes 2 mins to scrape langgraph

Scraping links
Choosing and preprocessing links
Classifying links and extracting nodes
Here's the complete code as a function:

```python
from typing import Annotated, Literal, TypedDict
from langchain_core.messages import HumanMessage
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

def create_weather_agent(anthropic_api_key: str):
    @tool
    def search(query: str):
        """Call to surf the web."""
        if "sf" in query.lower() or "san francisco" in query.lower():
            return "It's 60 degrees and foggy."
        return "It's 90 degrees and sunny."

    tools = [search]
    tool_node = ToolNode(tools)
    model = ChatAnthropic(api_key=anthropic_api_key, model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

    def should_continue(state: MessagesState) -> Literal["

In [42]:
# Test on meta_api fuctionals
from methods.meta_api import nodes_from_api
api_name = "langgraph"
doc_nodes, guide_nodes = nodes_from_api(api_name, fast_response=get_endpoint_response, slow_response=get_claude_response, max_links=5, view_on_frontend=True) # Takes 2 mins to scrape langgraph

Scraping links
Choosing and preprocessing links
Classifying links and extracting nodes
     :: Query time: 4.51s
ERROR PARSING CODE
     :: Evolution time: 16.33s
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        2
  ⏱️ Time breakdown:
     :: Query time: 4.51s
     :: Evolution time: 16.33s
     :: Evaluation time: 0.99s
     :: Total time: 21.83s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
No module named 'langchain.anthropic'


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
No module named 'langchain_anthropic'

     :: Query time: 4.58s
     :: Evolution time: 11.41s
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        3
  ⏱️ Time

Concurrent Execution of 1 Node functions to generate prompts ...: 100%|█| 1/1 [0


🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.00
  🔄 Compiled solutions:        1
  ⏱️ Time breakdown:
     :: Query time: 4.96s
     :: Evolution time: 17.49s
     :: Evaluation time: 1.90s
     :: Total time: 24.35s


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Must provide state_schema or input and output

     :: Query time: 4.04s
     :: Evolution time: 9.68s

Code 0 outputs:
Test 0: {'updated_state': ({'history': [{'event': 'tool_call'}, {'event': 'tool_call'}, ''], 'summary': '', 'pruned_items': []}, '', [])}
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 1.00
  🎯 Functional fitness: 0.00
  ⭐ Global fitness:     0.50
  🔄 Compiled solutions:        3
  ⏱️ Time breakdown:
     :: Query time: 4.04s
     :: Evolution time: 9.68s
     :: Evaluation time: 1.62s
     :: Total time: 15.34s


📊 Code 0: Fitness: 50.0%
-----------

In [ ]:
# Try to get into this documentation instead for better scrawling
# https://python.langchain.com/v0.1/docs/use_cases/web_scraping/

In [34]:
%%timeit -n 3 -r 1
import http
import json
import urllib
import os

from bs4 import BeautifulSoup
import httpx

api_name = "langgraph"

def _search_google(query: str):
    """
    Use Serper API to search Google for information

    Args:
        query (str): The search query

    Returns:
        Dict[str, Any]: Parsed JSON response from the API
    """
    conn = http.client.HTTPSConnection("google.serper.dev")
    payload = json.dumps({"q": query})
    headers = {
        "X-API-KEY": os.environ["SERPER_API_KEY"],
        "Content-Type": "application/json",
    }

    try:
        conn.request("POST", "/search", payload, headers)
        res = conn.getresponse()
        data = res.read()
        return json.loads(data.decode("utf-8"))
    except Exception as e:
        print(f"Error occurred during API request: {str(e)}")
        return {}
    finally:
        conn.close()


def get_content(link):
    res = httpx.get(link, follow_redirects=True)
    res.raise_for_status()
    url = str(res.url)

    return res.text, url

link = _search_google(f"{api_name} python docs")["organic"][0]["link"]
domain = urllib.parse.urlparse(link).netloc
try:
    content, url = get_content(link)
except httpx.HTTPStatusError as e:
    print(f"Error occurred during request: {str(e)}")

htmls = [content]
links = [url]
ptr = 0
while ptr < len(links):
    soup = BeautifulSoup(htmls[ptr], "html.parser")
    for a in soup.find_all("a", href=True):
        href = a["href"]
        new_link = urllib.parse.urljoin(links[ptr], href.split("#")[0])
        parse = urllib.parse.urlparse(new_link)
        if (
            links[0] in new_link
            and new_link not in links
            and new_link + "/" not in links
            and parse.scheme in ["http", "https"]
        ):
            try:
                content, url = get_content(new_link)
                htmls.append(content)
                links.append(url)
            except httpx.HTTPStatusError as e:
                print(f"Error occurred during request: {str(e)}")
    ptr += 1

42.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)


In [32]:
import time
import http
import json
import urllib
import asyncio
import os

from bs4 import BeautifulSoup
import httpx
api_name = "langgraph"
for _ in range(3):
    start_time = time.time()
    client = httpx.AsyncClient(timeout=None)

    def _search_google(query: str):
        """
        Use Serper API to search Google for information

        Args:
            query (str): The search query

        Returns:
            Dict[str, Any]: Parsed JSON response from the API
        """
        conn = http.client.HTTPSConnection("google.serper.dev")
        payload = json.dumps({"q": query})
        headers = {
            "X-API-KEY": os.environ["SERPER_API_KEY"],
            "Content-Type": "application/json",
        }

        try:
            conn.request("POST", "/search", payload, headers)
            res = conn.getresponse()
            data = res.read()
            return json.loads(data.decode("utf-8"))
        except Exception as e:
            print(f"Error occurred during API request: {str(e)}")
            return {}
        finally:
            conn.close()


    async def aget_content(link, client):
        res = await client.get(link, follow_redirects=True)
        if res.status_code == 200:
            url = str(res.url)
            return res.text, url
        else:
            return "", ""

    webpages = []
    visit = set()

    async def recursive_search(link, client, ref_url=None):
        content, url = await aget_content(link, client)
        if ref_url is None:
            ref_url = url
        if url and content:
            webpages.append((url, content))
            visit.add(url)
        soup = BeautifulSoup(content, "html.parser")
        new_links = []
        for a in soup.find_all("a", href=True):
            href = a["href"]
            new_link = urllib.parse.urljoin(link, href.split("#")[0])
            parse = urllib.parse.urlparse(new_link)
            if (
                ref_url in new_link
                and new_link not in visit
                and new_link + "/" not in visit
                and parse.scheme in ["http", "https"]
            ):
                visit.add(new_link)
                new_links.append(new_link)
        await asyncio.gather(*[recursive_search(link, client, ref_url) for link in new_links])
        

    link = _search_google(f"{api_name} python docs")["organic"][0]["link"]
    domain = urllib.parse.urlparse(link).netloc

    await recursive_search(link, client)

    await client.aclose()
    end = time.time()
    print(f"Time taken: {end - start_time}")
    print(f"Number of webpages: {len(webpages)}")

Time taken: 32.10880398750305
Number of webpages: 191
Time taken: 31.19189691543579
Number of webpages: 191
Time taken: 32.00479292869568
Number of webpages: 191


In [ ]:
# test case || evaluation || 
# - custom metric 
# - 